In [ ]:
# Install necessary libraries
#!pip install plotly pandas numpy

# Importing necessary libraries
import pandas as pd
import numpy as np
import plotly.graph_objects as go
from PIL import Image


In [ ]:
from google.colab import drive

# Mount Google Drive to access files
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
lunar_map_path = '/content/drive/My Drive/lunar_map_resized_big.png'
csv_path = '/content/drive/My Drive/new_map_df.csv'

In [ ]:
# Load lunar map image and get dimensions dynamically
lunar_map = Image.open(lunar_map_path)
img_width, img_height = lunar_map.size  # Dynamically get image width and height

print(f"Lunar map dimensions: Width = {img_width}, Height = {img_height}")


Lunar map dimensions: Width = 7500, Height = 3750


In [ ]:
import pandas as pd

# Load the entire dataset
df = pd.read_csv(csv_path)
print(f"Loaded {len(df)} points in the full dataset.")

# Randomly sample 1000 points
#df = df.sample(n=1000, random_state=42)  # Randomly sample 1000 points
#print(f"Randomly selected {len(df)} points for testing.")

# Ensure columns for latitude, longitude, and ratios exist
assert 'Latitude' in df.columns, "Latitude column not found!"
assert 'Longitude' in df.columns, "Longitude column not found!"
required_ratios = [
    "O/Si_ratio", "Na/Si_ratio", "Mg/Si_ratio", "Al/Si_ratio",
    "Ca/Si_ratio", "Ti/Si_ratio", "Mn/Si_ratio", "Fe/Si_ratio"
]
assert all(ratio in df.columns for ratio in required_ratios), "One or more ratio columns are missing!"

# Scale latitude and longitude to pixel coordinates
#left -180 right 180 up -90 down +90
#previous
#df['x_pixel'] = ((df['Longitude'] + 180) / 360) * img_width
#df['y_pixel'] = ((90 - df['Latitude']) / 180) * img_height  # Flip latitude for image alignment

#left -180 right 180 up 90 down -90
df['x_pixel'] = ((df['Longitude'] + 180) / 360) * img_width
df['y_pixel'] = ((df['Latitude']-90) / 180) * img_height  # Flip latitude for image alignment

print("Pixel coordinates calculated for latitude and longitude.")


Loaded 904908 points in the full dataset.
Pixel coordinates calculated for latitude and longitude.


In [ ]:
# Set scaling factor for figure size
scaling_factor = 0.4  # Adjust this as needed
fig_width = img_width * scaling_factor
fig_height = fig_width / (img_width / img_height)  # Maintain aspect ratio


In [ ]:
import plotly.graph_objects as go

# Create the Plotly figure with WebGL rendering
fig = go.Figure()

# Add the lunar map as the background
fig.add_layout_image(
    dict(
        source=lunar_map,
        x=0,
        y=img_height,
        xref="x",
        yref="y",
        sizex=img_width,
        sizey=img_height,
        xanchor="left",
        yanchor="top",
        layer="below",
    )
)

# Add scatter points using WebGL with hovertemplate and customdata
fig.add_trace(
    go.Scattergl(
        x=df['x_pixel'],
        y=df['y_pixel'],
        mode="markers",
        marker=dict(size=1, color='blue', opacity=0.25),
        hovertemplate=(
            "Lat: %{customdata[0]:.2f}<br>"
            "Lon: %{customdata[1]:.2f}<br>"
            "O/Si_ratio: %{customdata[2]}<br>"
            "Na/Si_ratio: %{customdata[3]}<br>"
            "Mg/Si_ratio: %{customdata[4]}<br>"
            "Al/Si_ratio: %{customdata[5]}<br>"
            "Ca/Si_ratio: %{customdata[6]}<br>"
            "Ti/Si_ratio: %{customdata[7]}<br>"
            "Mn/Si_ratio: %{customdata[8]}<br>"
            "Fe/Si_ratio: %{customdata[9]}<extra></extra>"
        ),
        customdata=df[[
            "Latitude", "Longitude", "O/Si_ratio", "Na/Si_ratio", "Mg/Si_ratio",
            "Al/Si_ratio", "Ca/Si_ratio", "Ti/Si_ratio", "Mn/Si_ratio", "Fe/Si_ratio"
        ]].values  # Pass raw data for hover
    )
)

# Correct aspect ratio and layout
aspect_ratio = img_width / img_height  # Calculate aspect ratio of the image

# Set figure width and height to maintain correct scaling
fig_width = img_width * 0.4
fig_height = fig_width / aspect_ratio

# Update layout for scaling and alignment
fig.update_layout(
    width=fig_width,
    height=fig_height,
    xaxis=dict(range=[0, img_width], visible=False),  # Match x-axis to image width
    yaxis=dict(range=[0, img_height], visible=False),  # Match y-axis to image height
    title="Interactive Lunar Map with Ratios",
)

# Lock axes scaling to maintain alignment
fig.update_yaxes(
    scaleanchor="x",  # Lock y-axis scaling to x-axis
    scaleratio=1      # 1:1 ratio for x and y
)

# Save the figure as an HTML file
output_file_path = '/content/drive/My Drive/interactive_trial_lunar_map.html'
fig.write_html(output_file_path)

print(f"Interactive map saved at: {output_file_path}")


Interactive map saved at: /content/drive/My Drive/new_trial_lunar_map_interactive_web_hovertemplate.html
